## **Importing All the Modules**

- **`YOLO` (from `ultralytics`)**
  - Load and run the **YOLOv8 model**.

- **`cv2`**
  - Read/write images.
  - Draw bounding boxes and other shapes.
  - Perform color space conversions (e.g., BGR ↔ RGB).

- **`numpy as np`**
  - Work with image arrays.
  - Perform numeric and matrix operations efficiently.

- **`typing.List` / `typing.Dict`**
  - Add **type annotations** for better code clarity and readability.

- **`torch`**
  - Detect GPU availability using `torch.cuda.is_available()`.
  - Perform tensor operations if needed.

- **`os`**
  - Read environment variables.
  - Check file paths and manage the filesystem.

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from typing import List, Dict
import torch

### Loading Environment Variables

```python
# Load environment variables from a .env file


In [2]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="D:\PycharmProjects\Eco_Vision\Backend\.env")

<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
C:\Users\HP\AppData\Local\Temp\ipykernel_8852\2373333911.py:3: SyntaxWarning: invalid escape sequence '\P'
  load_dotenv(dotenv_path="D:\PycharmProjects\Eco_Vision\Backend\.env")


True

### YOLO Model Setup

- **Device Selection:**
  You can **force the use of CPU or GPU** manually.
  If not specified, the device will be **auto-detected** later.

- **Model Path (`model_path`):**
  - Uses the environment variable `MODEL_PATH` if it exists.
  - Otherwise, defaults to `"yolov8n.pt"`.

- **`YOLO(model_path)`**:
  - Calls the **YOLO class** from Ultralytics with the given `model_path`.
  - Loads the **pretrained model into memory**.
  - **Important:** The model is loaded **once in the constructor**, so you **don’t need to reload it** every time you perform detection.


In [3]:
model_path = os.getenv("MODEL_PATH",None) #if we didn't set model , default this come "yolo12n.pt"
device = ("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO(model_path)

#### As here  below we can see our model is lock & loaded on cpu not gpu as i dont have didicated graphic card 💀.

#### we are using very small model(nano ones) here and it it fatest , but have low accuracy

In [4]:
print(f"[YOLODetector] Loaded model: {model_path} on {device}")

[YOLODetector] Loaded model: yolo12n.pt on cpu
